In [1]:
# import essential libraries
import PyPDF2
import re

In [5]:
# extract text from pages
fhand = open('D:\GitHub_Projects\gybn-pb-insights\dataset\PolicyBrief_Nature-based-Solutions.pdf', 'rb')
file_data = PyPDF2.PdfFileReader(fhand, strict=False)
print('Total pages: ', file_data.numPages)

Total pages:  18


In [10]:
page_range = list(range(3, 12))
raw_data = []
total_chars = []
for page in page_range:
    data = file_data.getPage(page).extractText()
    raw_data.append(data)
    total_chars.append(len(data))

print('Total Characters in Report: ',sum(total_chars))

Total Characters in Report:  24803


In [21]:
# 1st page 1st line word-counter Test
page1_1 = raw_data[0].split('\n')[0].split()
count_dict = {}
for word in page1_1:
    count_dict[word] = count_dict.get(word, 0) + 1
count_dict

{'3The': 1,
 'term': 1,
 'ﬁNature-based': 1,
 'Solutionsﬂ': 1,
 '(NbS)': 1,
 'started': 1,
 'being': 2,
 'used': 1,
 'in': 1,
 'the': 1,
 'late': 1,
 '2000s,': 1,
 '˜rst': 1,
 'put': 1,
 'forward': 1,
 'by': 1,
 'practitioners': 1,
 'as': 1}

In [44]:
# 1st page all lines word-counter Test
page1 = raw_data[0].split('\n')
page1 = [word for sent in page1 for word in sent.split() if word != '']
counter_dict = {}
for word in page1:
    counter_dict[word] = counter_dict.get(word, 0) + 1
print(f"Common Word : {list(counter_dict.keys())[list(counter_dict.values()).index(max(counter_dict.values()))]}, Counter : {max(counter_dict.values())}")

Common Word : and, Counter : 22
